In [1]:
import re
import os
import cv2
import numpy as np
from scipy import misc
from skimage import feature
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report,confusion_matrix

In [2]:
#defining Local Binary Patterns

class LocalBinaryPatterns:
    
    def __init__(self,num_points,radius):
        self.numpoints=num_points
        self.radius=radius
        
    def describe(self,image,eps=1e-7):
        lbp=feature.local_binary_pattern(image,self.numpoints,self.radius,method="uniform")
        new_lbp=lbp.astype(dtype='uint8')
        #print(new_lbp)
        #cv2.imshow("lbp image",new_lbp)
        #cv2.waitKey(0)
        (hist,_)=np.histogram(lbp.ravel(),bins=np.arange(0,self.numpoints+3),range=(0,self.numpoints+2))
        
        hist=hist.astype("float")
        hist/=(hist.sum()+eps)
        
        return hist

In [50]:
'''
im=cv2.imread(r'C:\Users\Legion\Documents\standard_test_images\mandril_gray.tif',0)
func=LocalBinaryPatterns(150,10)
h1=func.describe(im)

plt.figure()
plt.plot(h1)
plt.show()
'''


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 19-20: truncated \UXXXXXXXX escape (<ipython-input-50-5f99ff308cdc>, line 9)

In [4]:
desc=LocalBinaryPatterns(24,8)
data=[]
labels=[]

main_folder_path=r'C:\Users\Legion\Desktop\Internship_IIT\GRAZ'
segments=[r'bikes\bikes',r'persons\persons']

for seg in segments:
    absolute_path=os.path.join(main_folder_path,seg)
    
    for image_name in os.listdir(absolute_path):
        original__grey_image=misc.imread(os.path.join(absolute_path,image_name),mode='L')

        h1=desc.describe(original__grey_image)
        
        z=re.match("^bike.",image_name)
        if z:
            labels.append(0)
        else:
            labels.append(1)
            
        data.append(h1)
        


C:\Users\Legion\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  if sys.path[0] == '':


In [6]:
data

AttributeError: 'list' object has no attribute 'shape'

In [7]:
data=np.asarray(data,dtype=float)
labels=np.asarray(labels,dtype=int)

#shuffling the feature vectors and labels together
randomise=np.arange(len(data))
np.random.shuffle(randomise)
data=data[randomise]
labels=labels[randomise]


#trainand test splitting
data_train, data_test, labels_train, labels_test = train_test_split(data, labels, test_size=0.2)

In [8]:
data_train.shape

(666, 26)

In [59]:
#linear SVM
linearsvclassifier =SVC(kernel='linear')
linearsvclassifier.fit(data_train,labels_train)

labels_pred=linearsvclassifier.predict(data_test)

print(confusion_matrix(labels_test,labels_pred))
print(classification_report(labels_test,labels_pred))
print("Accuracy : ",metrics.accuracy_score(labels_test,labels_pred))

[[40 27]
 [21 79]]
              precision    recall  f1-score   support

           0       0.66      0.60      0.62        67
           1       0.75      0.79      0.77       100

   micro avg       0.71      0.71      0.71       167
   macro avg       0.70      0.69      0.70       167
weighted avg       0.71      0.71      0.71       167

Accuracy :  0.7125748502994012


In [10]:
#kernel SVM
kernelsvclassifier =SVC(kernel='rbf')
kernelsvclassifier.fit(data_train,labels_train)

labels_pred=kernelsvclassifier.predict(data_test)

print(confusion_matrix(labels_test,labels_pred))
print(classification_report(labels_test,labels_pred))
print("Accuracy : ",metrics.accuracy_score(labels_test,labels_pred))

[[ 0 70]
 [ 0 97]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        70
           1       0.58      1.00      0.73        97

   micro avg       0.58      0.58      0.58       167
   macro avg       0.29      0.50      0.37       167
weighted avg       0.34      0.58      0.43       167

Accuracy :  0.5808383233532934


C:\Users\Legion\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Legion\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
